# Model 1 - MCMC with Turing.jl

In [16]:
using Distributions, StatsFuns
using Mamba: describe, plot

using Turing
Pkg.checkout("Turing", "master") 

INFO: Checking out Turing master...
INFO: Pulling Turing latest master...
INFO: Cloning cache of DocStringExtensions from https://github.com/JuliaDocs/DocStringExtensions.jl.git
INFO: Cloning cache of Documenter from https://github.com/JuliaDocs/Documenter.jl.git
INFO: Cloning cache of Stan from https://github.com/goedman/Stan.jl.git
INFO: Installing DocStringExtensions v0.4.0
INFO: Installing Documenter v0.11.4
INFO: Installing Stan v2.0.1
INFO: Building Homebrew
INFO: Building Stan


## STEP 1 - Define data

In [20]:
data = Dict()
data["T"] = 100
data["k"] = [50, 51, 57, 55, 63, 62, 82, 94, 99, 100]
data["Δμ"] = [0.0100, 0.0215, 0.0464, 0.1000, 0.2154, 0.4642, 1.0000, 2.1544, 4.6416, 10.0000];

## STEP 2 - Parameter recovery

First we are going to define a model. This is the way of doing it for the Turing package. I managed to convert this over from the JAGS specification (see the original Matlab repo) with not so much trouble. It's pretty nice in fact.

In [21]:
@model model1(k, Δμ, T) = begin
    Φ(x) = normcdf.(0, 1, x)
    PC(Δμ, σ²) = Φ(Δμ/sqrt(2σ²))
    σ² ~ Uniform(0,100)
    for c = 1:length(k)
        k[c]       ~ rand( Binomial(T, PC(Δμ[c],σ²)) )
        #k_pred[c] = rand( Binomial(T, PC(Δμ[c],σ²)) )
    end
    return k#, k_pred
end

# TODO: get posterior samples of k (called k_pred)

model1 (generic function with 4 methods)

## Invoke sampling
We will simultanously sample from the posterior distribution over $\sigma^2$ as well as the posterior predictive

In [22]:
samples = sample( model1(data["k"],data["Δμ"], data["T"]), PG(50,300));

Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] exp(::Array{Float64,1}) at ./deprecated.jl:57
 [3] weights(::Turing.ParticleContainer{Turing.Traces.Trace{:R}}) at /Applications/JuliaPro-0.6.0.1.app/Contents/Resources/pkgs-0.6.0.1/v0.6/Turing/src/core/container.jl:130
 [4] consume(::Turing.ParticleContainer{Turing.Traces.Trace{:R}}) at /Applications/JuliaPro-0.6.0.1.app/Contents/Resources/pkgs-0.6.0.1/v0.6/Turing/src/core/container.jl:94
 [5] step(::Function, ::Turing.Sampler{Turing.PG}, ::Turing.VarInfo) at /Applications/JuliaPro-0.6.0.1.app/Contents/Resources/pkgs-0.6.0.1/v0.6/Turing/src/samplers/pgibbs.jl:67
 [6] macro expansion at ./util.jl:293 [inlined]
 [7] #sample#85(::Bool, ::Void, ::Int64, ::Function, ::Function, ::Turing.PG) at /Applications/JuliaPro-0.6.0.1.app/Contents/Resources/pkgs-0.6.0.1/v0.6/Turing/src/samplers/pgibbs.jl:114
 [8] sample(::Function, ::Turing.PG) at /Applications/JuliaPro-0.6.0.1.app/Contents/Resources/pkgs-0.6.0.1/v0.6/Turing/src/

LoadError: [91mUndefVarError: σ² not defined[39m

# Visualisation
Plot data and posterior predictions in data space

Plot posterior in parameter space

In [ ]:
describe(samples)


In [6]:
p1 = plot(samples)

## References
Vincent, B. T. (2015). A tutorial on Bayesian models of perception. Journal of Mathematical Psychology, 66, 103–114. http://doi.org/10.1016/j.jmp.2015.02.001